# instalación de paquetes y verificación

In [1]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [3]:
# vamos a instalar todos los paquetes necesarios para el funcionamiento de este cuaderno. Este cuaderno, junto con el
# repo de github con las carpetas y archivos nos ahorran tener que seguir el proceso descrito aqu: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1.md
# y además nos evitan posibles problemas
!pip install -r requirements.txt

ERROR: Invalid requirement: './object-detection' (from line 29 of requirements.txt)
Hint: It looks like a path. File './object-detection' does not exist.


In [6]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# creación del dataset

In [8]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images -o images -tr 0.1 -vr 0.15
!tree -d images

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images
├── test
├── train
└── val

3 directories


In [9]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py images/train images/train/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/test images/test/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/val images/val/coco --labels training/labels.txt
!tree -d images

Creating dataset: images/train/coco
Generating dataset from: images/train/images_frame2000.json
Generating dataset from: images/train/images_frame8050.json
Generating dataset from: images/train/images_frame3100.json
Generating dataset from: images/train/images_frame2750.json
Generating dataset from: images/train/images_frame7900.json
Generating dataset from: images/train/images_frame5800.json
Generating dataset from: images/train/images_frame10250.json
Generating dataset from: images/train/images_frame5300.json
Generating dataset from: images/train/images_frame2950.json
Generating dataset from: images/train/images_frame3750.json
Generating dataset from: images/train/images_frame10150.json
Generating dataset from: images/train/images_frame1700.json
Generating dataset from: images/train/images_frame5150.json
Generating dataset from: images/train/images_frame10050.json
Generating dataset from: images/train/images_frame7350.json
Generating dataset from: images/train/images_frame1350.json
G

Generating dataset from: images/val/images_frame8800.json
Generating dataset from: images/val/images_frame8400.json
Generating dataset from: images/val/images_frame1650.json
Generating dataset from: images/val/images_frame9450.json
Generating dataset from: images/val/images_frame1200.json
Generating dataset from: images/val/images_frame7750.json
Generating dataset from: images/val/images_frame5500.json
Generating dataset from: images/val/images_frame2900.json
Generating dataset from: images/val/images_frame10000.json
Generating dataset from: images/val/images_frame4300.json
Generating dataset from: images/val/images_frame3000.json
Generating dataset from: images/val/images_frame9150.json
Generating dataset from: images/val/images_frame2150.json
Generating dataset from: images/val/images_frame9550.json
Generating dataset from: images/val/images_frame4100.json
Generating dataset from: images/val/images_frame4450.json
Generating dataset from: images/val/images_frame2700.json
Generating da

<img src="training/ejemplo_labels_txt.PNG">

In [10]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm images/test/*
!mv images/test/coco/* images/test
!rm -rf images/test/coco 

!rm images/train/*
!mv images/train/coco/* images/train
!rm -rf images/train/coco 

!rm images/val/*
!mv images/val/coco/* images/val
!rm -rf images/val/coco 

!tree -d images

rm: cannot remove 'images/test/coco': Is a directory
rm: cannot remove 'images/train/coco': Is a directory
rm: cannot remove 'images/val/coco': Is a directory
images
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [11]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir=images/train \
--val_image_dir=images/val \
--test_image_dir=images/test \
--train_annotations_file=images/train/annotations.json \
--val_annotations_file=images/val/annotations.json \
--testdev_annotations_file=images/test/annotations.json \
--output_dir=training/tfrecord \
--include_masks=True

I0903 07:02:14.021196 140190018115392 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0903 07:02:14.021518 140190018115392 create_coco_tf_record.py:412] 0 images are missing annotations.
I0903 07:02:14.021673 140190018115392 create_coco_tf_record.py:441] On image 0 of 117
I0903 07:02:37.000346 140190018115392 create_coco_tf_record.py:441] On image 100 of 117
I0903 07:02:41.302762 140190018115392 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0903 07:02:41.307433 140190018115392 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0903 07:02:41.307577 140190018115392 create_coco_tf_record.py:412] 0 images are missing annotations.
I0903 07:02:41.307616 140190018115392 create_coco_tf_record.py:441] On image 0 of 24
I0903 07:02:46.662210 140190018115392 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0903 07:02:46.665430 140190018115392 create_coco_tf_record.py:39

# Entrenamiento y exportación del modelo

In [16]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training
!python3 object_detection/model_main.py --alsologtostderr \
--model_dir=training/ \
--pipeline_config_path=training/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

W0903 07:16:13.516900 140078348080960 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I0903 07:16:13.517046 140078348080960 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0903 07:16:13.517093 140078348080960 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0903 07:16:13.517131 140078348080960 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0903 07:16:13.517168 140078348080960 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0903 07:16:13.517218 140078348080960 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

Instructions for updating:
Use `tf.cast` instead.
W0903 07:16:26.169020 140078348080960 deprecation.py:323] From /home/bigdata/Documentos/proyecto_mario/tensorflow_object_detection_guide/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
^C
Traceback (most recent call last):
  File "object_detection/model_main.py", line 108, in <module>
    tf.app.run()
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "object_detection/model_main.py", line 104, i

In [13]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir training

checkpoint
ejemplo_labels_txt.PNG
eval_0
events.out.tfevents.1599116611.bigdata-alineware
faster_rcnn_inception_v2_coco.config
graph.pbtxt
label_map.pbtxt
labels.txt
mask_rcnn_inception_v2_coco.config
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-2744.data-00000-of-00001
model.ckpt-2744.index
model.ckpt-2744.meta
tfrecord


In [14]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob("training/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
print("En el campo trained_checkpoint debes poner lo siguiente: \ntraining/model.ckpt-"+str(max_step))

En el campo trained_checkpoint debes poner lo siguiente: 
training/model.ckpt-2744


In [15]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido de la fecha y la hora
!python3 object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training/model.ckpt-2744\
--output_directory inference_graph_02_09_10_14

Instructions for updating:
Please use `layer.__call__` method instead.
W0903 07:15:05.191840 140518929565504 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0903 07:15:06.003338 140518929565504 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0903 07:15:06.093859 140518929565504 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0903 07:15:06.094101 140518929565504 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0903 07:15:06.127783 14

242 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz

2020-09-03 07:15:09.197737: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-03 07:15:09.311350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:17:00.0
2020-09-03 07:15:09.312057: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:65:00.0
2020-09-03 07:15:09.347993: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-09-03 07:15:09.351125: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-09-03 07:15:09.353462: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0903 07:15:12.654336 140518929565504 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/tools/freeze_graph.py:233: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
W0903 07:15:12.654532 140518929565504 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 360 variables.
I0903 07:15:13.0834

In [42]:
!dir inference_graph_02_09_10_14

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config
